In [62]:
import re
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from msedge.selenium_tools import Edge, EdgeOptions 

In [86]:
def get_tweet_data(card):
    """Extract data from tweet card"""
    username = card.find_element_by_xpath('.//span').text
    try:
        handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    
    content = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text

    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    interactions_cnt = card.find_element_by_xpath('//a[@class="css-4rbku5 css-18t94o4 css-1dbjc4n r-1loqt21 r-1777fci r-bt1l66 r-1ny4l3l r-bztko3 r-lrvibr"]').text
    
    tweet = (username, handle, postdate, content, reply_cnt, retweet_cnt, like_cnt, interactions_cnt)
    return tweet 

In [87]:


# create instance of web driver
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options)



##################################

##driver.find_element_by_xpath('class="css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0"').click()


##password = driver.find_element_by_xpath("//in")

driver.get('https://twitter.com/login')

sleep(7)

username = driver.find_element_by_xpath("//input[@name='text']")
username.send_keys('scarp65839')


nextB = driver.find_element_by_xpath('//div[@class="css-18t94o4 css-1dbjc4n r-sdzlij r-1phboty r-rs99b7 r-ywje51 r-usiww2 r-2yi16 r-1qi8awa r-1ny4l3l r-ymttw5 r-o7ynqc r-6416eg r-lrvibr r-13qz1uu"]').click()
sleep(5)

password = driver.find_element_by_xpath('//input[@name="password"]')
password.send_keys('ICD.2023')
sleep(2)

nextB = driver.find_element_by_xpath('//div[@class="css-1dbjc4n r-pw2am6"]/div').click()

C:\Users\elix\AppData\Local\Temp\ipykernel_18932\2214035702.py:4: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(options=options)


In [88]:
search_term = input('search term: ')
# navigate to login screen
driver.get('https://twitter.com/search')
driver.maximize_window()
sleep(5)

# find search input and search for term
search_input = driver.find_element_by_xpath('//input[@aria-label="Search query"]')
search_input.send_keys(search_term)
search_input.send_keys(Keys.RETURN)
sleep(1)

# navigate to historical 'latest' tab
driver.find_element_by_link_text('Latest').click()

search term: Milei


In [89]:
# get all tweets on the page
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = curr_position
            break

# close the web driver
driver.close()

In [90]:
with open('Milei.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['DisplayName', 'UserName', 'Timestamp', 'Content', 'Comments', 'Likes', 'Retweets','Interactions']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)

In [121]:
import pandas as pd
import numpy as np

df = pd.read_csv("delitos.csv")


## LIMPIEZA

In [122]:
df.dropna(subset=["Content"], inplace=True)

In [123]:
df

,DisplayName,UserName,Timestamp,Content,Comments,Likes,Retweets,Interactions,Delito,Coordenadas
0,Segura EP,@segura_ep,2023-09-01T03:20:23.000Z,Desde las cámaras de videovigilancia observamo...,NaN,2.000,8,"2,314",1,"-2.1384511, -79.8913772"
1,Segura EP,@segura_ep,2023-09-01T02:19:12.000Z,Desde las cámaras de videovigilancia captamos ...,NaN,1.000,7,"2,314",1,"-2.230788, -79.8939582"
2,Segura EP,@segura_ep,2023-08-31T01:53:39.000Z,Desde las cámaras de videovigilancia realizamo...,2.0,3.000,6,"1,335",1,"-2.2438513, -79.8956694"
3,Segura EP,@segura_ep,2023-08-29T17:10:33.000Z,Un vehículo reportado como robado fue captado ...,NaN,6.000,20,"2,499",1,"-2.1749333, -79.9417115"
4,Segura EP,@segura_ep,2023-08-29T08:04:47.000Z,#Alerta | Miembros de la \n@ATM_Transito\n y \...,1.0,5.000,13,"2,499",1,"-3.9952065, -79.2022123"
...,...,...,...,...,...,...,...,...,...,...
547,Minuto & Medio,@MinMedio,2023-07-16 18:48:04,#Guayaquil _x000D_\nChoferes de buses urbanos ...,44.0,122.000,277,28.3K,1,NaN
548,Minuto & Medio,@MinMedio,2023-07-17 16:45:26,"Como en la década de los 90, vendedores se tom...",634.0,1.135,"1,999",73K,1,NaN
549,Minuto & Medio,@MinMedio,2023-07-18 21:43:43,#Guayaquil _x000D_\nPolicia nacional detuvo a ...,NaN,18.000,124,48K,1,"-2.101462813094633, -79.93364728590731"
550,Minuto & Medio,@MinMedio,2023-07-19 08:04:06,#Guayaquil_x000D_\n3 buses de la metrovia se i...,5.0,34.000,116,48K,1,"-2.1391408737042266, -79.87940332154923"


In [124]:
def str_to_int_milesimas(value):
    if isinstance(value, str):
        value = value.split('\r')[0]  # Eliminar todo después de '\r'
        value = ''.join(filter(lambda char: char.isdigit() or char.upper() in ['K', 'M'], value))
        if value.endswith('K'):
            value = value[:-1]
            value = int(value) * 1000
        elif value.endswith('M'):
            value = value[:-1]
            value = int(value) * 1000000
        else:
            value = int(value)
        return int(value)
    return value

In [125]:
def get_real_count(value):
    if 'K' in value:
        return str(float(value.replace('K', '')) * 1000)
    if 'M' in value:
        return str(float(value.replace('M', '')) * 1000000)
    return value

In [126]:
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

In [127]:
df['Comments'] = df['Comments'].fillna(0)

In [128]:
df['Likes'] = df['Likes'].apply(str_to_int_milesimas)
df['Likes'] = df['Likes'].fillna(0)

In [129]:
df['Retweets'] = df['Retweets'].apply(str_to_int_milesimas)
df['Retweets'] = df['Retweets'].fillna(0)

In [130]:
df['Interactions'] = df['Interactions'].apply(str_to_int_milesimas)
df['Interactions'] = df['Interactions'].fillna(0)

In [131]:
df

,DisplayName,UserName,Timestamp,Content,Comments,Likes,Retweets,Interactions,Delito,Coordenadas
0,Segura EP,@segura_ep,2023-09-01 03:20:23+00:00,Desde las cámaras de videovigilancia observamo...,0.0,2.000,8.0,2314.0,1,"-2.1384511, -79.8913772"
1,Segura EP,@segura_ep,2023-09-01 02:19:12+00:00,Desde las cámaras de videovigilancia captamos ...,0.0,1.000,7.0,2314.0,1,"-2.230788, -79.8939582"
2,Segura EP,@segura_ep,2023-08-31 01:53:39+00:00,Desde las cámaras de videovigilancia realizamo...,2.0,3.000,6.0,1335.0,1,"-2.2438513, -79.8956694"
3,Segura EP,@segura_ep,2023-08-29 17:10:33+00:00,Un vehículo reportado como robado fue captado ...,0.0,6.000,20.0,2499.0,1,"-2.1749333, -79.9417115"
4,Segura EP,@segura_ep,2023-08-29 08:04:47+00:00,#Alerta | Miembros de la \n@ATM_Transito\n y \...,1.0,5.000,13.0,2499.0,1,"-3.9952065, -79.2022123"
...,...,...,...,...,...,...,...,...,...,...
547,Minuto & Medio,@MinMedio,2023-07-16 18:48:04,#Guayaquil _x000D_\nChoferes de buses urbanos ...,44.0,122.000,277.0,283000.0,1,NaN
548,Minuto & Medio,@MinMedio,2023-07-17 16:45:26,"Como en la década de los 90, vendedores se tom...",634.0,1.135,1999.0,73000.0,1,NaN
549,Minuto & Medio,@MinMedio,2023-07-18 21:43:43,#Guayaquil _x000D_\nPolicia nacional detuvo a ...,0.0,18.000,124.0,48000.0,1,"-2.101462813094633, -79.93364728590731"
550,Minuto & Medio,@MinMedio,2023-07-19 08:04:06,#Guayaquil_x000D_\n3 buses de la metrovia se i...,5.0,34.000,116.0,48000.0,1,"-2.1391408737042266, -79.87940332154923"


In [114]:
df = df.loc[~df["Content"].str.contains("Replying")]

In [115]:
df

,DisplayName,UserName,Timestamp,Content,Comments,Likes,Retweets,Interactions
5,Ámbito Financiero,@Ambitocom,2023-09-02 17:49:26+00:00,"Pablo Moyano, sobre un eventual triunfo de Bul...",0.0,0.0,0.0,0.0
8,Martyn Mcfly,@martinmpaz,2023-09-02 17:49:21+00:00,"La ""estrategia"" parece ser:\nSeguir castigando...",0.0,0.0,0.0,0.0
10,+Contenidos.Net,@ContenidosN,2023-09-02 17:49:12+00:00,Milei acusó a Pallarols de mentiroso y si lleg...,0.0,0.0,0.0,0.0
11,OLGAOLGA,@OLGAOLGA577671,2023-09-02 17:49:09+00:00,El panqueque y cia. sabe que con Pato no va a ...,0.0,0.0,0.0,0.0
17,Pat Wermus,@PatWermus,2023-09-02 17:48:52+00:00,Milei es un invento de Fantino y el grupo Amér...,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
725,Raskol,@RaskolBC,2023-09-02 17:23:51+00:00,"Es evidente, de ganar Milei habrá una relectur...",1.0,5.0,26.0,180.0
731,Rubén de Rosende,@trodain,2023-09-02 17:23:44+00:00,"Xulio Abonjo, Manu Carreño, Adam Driver y Javi...",0.0,0.0,0.0,180.0
736,Rubén de Rosende,@trodain,2023-09-02 17:23:44+00:00,"Xulio Abonjo, Manu Carreño, Adam Driver y Javi...",0.0,0.0,0.0,9.0
738,Félix Villarroel.,@felixvillarro14,2023-09-02 17:23:41+00:00,Nivel Maria Corina Machado osea nivel locura t...,0.0,8.0,6.0,9.0


In [116]:
df.to_csv('Milei_limpio.csv', index=False)

In [117]:
df_1 = pd.read_csv("Milei_limpio.csv")
df_2 = pd.read_csv("NetflixES_limpio.csv")
df_3 = pd.read_csv("QuantumFracture_limpio.csv")

In [118]:
df_no_delitos = pd.concat( [df_1, df_2, df_3],ignore_index=True)

In [134]:

df.to_csv('delitos_limpio.csv', index=False)

In [139]:
df_no = pd.read_csv("no_delitos.csv")

In [140]:
df_no

,DisplayName,UserName,Timestamp,Content,Comments,Likes,Retweets,Interactions
0,Ámbito Financiero,@Ambitocom,2023-09-02 17:49:26+00:00,"Pablo Moyano, sobre un eventual triunfo de Bul...",0.0,0.0,0.0,0.0
1,Martyn Mcfly,@martinmpaz,2023-09-02 17:49:21+00:00,"La ""estrategia"" parece ser:\nSeguir castigando...",0.0,0.0,0.0,0.0
2,+Contenidos.Net,@ContenidosN,2023-09-02 17:49:12+00:00,Milei acusó a Pallarols de mentiroso y si lleg...,0.0,0.0,0.0,0.0
3,OLGAOLGA,@OLGAOLGA577671,2023-09-02 17:49:09+00:00,El panqueque y cia. sabe que con Pato no va a ...,0.0,0.0,0.0,0.0
4,Pat Wermus,@PatWermus,2023-09-02 17:48:52+00:00,Milei es un invento de Fantino y el grupo Amér...,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
525,Wild Project,@datos_twp,2023-07-10 09:36:44+00:00,"Tras el Especial Verano...\n\n Crespo, \n@Qua...",1.0,3.0,90.0,455.0
526,EzE,@Ezequie28330501,2023-07-10 08:26:49+00:00,"quantumfracture, un representante del cientifi...",1.0,0.0,0.0,455.0
527,♰ 𝐂𝐀𝐓𝐓𝐔𝐒 𝐈𝐈𝐈 ♰,@Cattus_3,2023-07-09 23:18:39+00:00,Respecto del argumento final que da quantumfra...,0.0,1.0,1.0,455.0
528,♰ 𝐂𝐀𝐓𝐓𝐔𝐒 𝐈𝐈𝐈 ♰,@Cattus_3,2023-07-09 22:09:17+00:00,Cada vez que quantumfracture pone el h0locaust...,1.0,0.0,1.0,455.0


In [141]:
df_no['Delito'] = 0

In [143]:

df_no.to_csv('no_delitos_limpio.csv', index=False)